
### Kyriba Open Report Studio PoC

We expect that the user will build a report in excel with the desired visual output and all intermediate computation

Data From Kyriba Standard reports will be stored in each dedicated sheet
Sheet will be named "KYR xxxx" where xxx is the PT name defined in Kyriba
Ouput of each PT need to be set as xlxs in Kyriba

To run:
- Upload config.csv containing API credentials on the left side (Files icon)
- Go in the menu Runtime -> Run All
- Drag and drop or upload the excel template (the code will pause until a file is uploaded)
- Each PT wil executed 
- Excel will generated from the template, with each PT tabs refreshed
- Report will be generated and automatically downloaded




## <font color='red'>Code for token generation and API management </font>





In [1]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO" #@param ['DEMO', '22SP9', '22SP8', '22SP7'] {allow-input: true}

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ") 
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git") 
import importlib

enableFormatterDF = True #@param {type:"boolean"}
from google.colab import data_table

if enableFormatterDF is True:
  data_table.enable_dataframe_formatter()
else:
  data_table.disable_dataframe_formatter()

try:  
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests

main



In [2]:
#@title Import custom modules
import pandas as pd
from io import StringIO
from google.colab import files
import json as js
from zipfile import ZipFile
import time , requests, os.path, openpyxl , os, glob
from datetime import datetime
import copy
from openpyxl.utils import get_column_letter
from openpyxl.styles import  PatternFill, Font,  Alignment

In [3]:
#@title <font color='lime'>Connect and get token</font>
print('token:')
token = sample_requests.login()


token:
6af8f48e-a4f6-4955-a81d-45bc5c0f51eb


In [4]:
#@title List of Process Templates listed in the excel file
 #Load and get Sheet'names from Report Template

uploaded = files.upload()
for fn in uploaded.keys():
  new_template = fn
  
wb_load = openpyxl.load_workbook(new_template)
list_of_sheets = wb_load.sheetnames

 #Scan name of Sheets
match_words = [s for s in list_of_sheets if "KYR" in s]

 # Get tempalte corespond with KYR
new_list = [s.replace("KYR ", "") for s in match_words]
print ("List of template: ")
i = 0
while i < len(new_list):
    w= new_list[i]
    print (w)
    i += 1

Saving UseCase Template.xlsx to UseCase Template (1).xlsx
List of template: 
DEMOREPORT1
DEMOREPORT2
DEMOREPORT3
NoData
ET


## <font color='red'>Code for generating PTs </font>


In [ ]:
#@title Process Templates generator { form-width: "20%" }


platform = "demo.kyriba.com" #@param ["demo.kyriba.com", "gemini.proto.cloud.kyriba.com", "leo.proto.cloud.kyriba.com", "ruby-preprod.treasury-factory.com", "ruby-sandbox.treasury-factory.com", "ruby.treasury-factory.com", "sapphire-preprod.treasury-factory.com", "sapphire-sandbox.treasury-factory.com", "sapphire.treasury-factory.com", "topaz-preprod.treasury-factory.com", "topaz-sandbox.treasury-factory.com", "topaz.treasury-factory.com", "emerald-preprod.treasury-factory.com", "emerald-sandbox.treasury-factory.com", "emerald.treasury-factory.com", "diamond-preprod.treasury-factory.com", "diamond-sandbox.treasury-factory.com", "diamond.treasury-factory.com", "opal-preprod.treasury-factory.com", "opal-sandbox.treasury-factory.com", "opal.treasury-factory.com"]
baseURL = 'https://{}/gateway/api'.format(platform);


Stat_data = []

#Loop to generate PTs
i = 0
while i < len(new_list):
    w= new_list[i] 

    Reportfile = "KYR "+ w +".xlsx";
    start_time= time.time()
#Run KYR and get Task Id 
    response = sample_requests.post_results(token,"/v1/process-templates/"+ w +"/run",{},{})

#No response returned 
    if not (response):
      print('Process Template', w, "not found")        
      Stat_data.append([w,"Not Found",format(time.time() - start_time, ".0f")])
    
    else: 

# taskId returned
      taskId = response[0]['taskId']
          
#Run PT and get task status
      url = "/v1/process-templates/"+taskId+"/status"
      time.sleep(2)
      response = sample_requests.get_results(token, url)   
      status = js.loads(response)['status']
     
      

      if status == "Complete":
        Stat_data.append([w,status,format(time.time() - start_time, ".0f")])
        print("\r","Generating file", w , ".", "Task status :", status,".","Time elapsed in seconds :",  format(time.time() - start_time, ".0f"),".",end="")

#Run PT and get file content
        url = baseURL + "/v1/process-templates/"+w+"/files?taskId="+taskId
        headers = {
        'Authorization': 'Bearer '+token
  }
        response = requests.get(url, headers=headers, data={})
        print("\r","Generating file", w ,"." ,"Task status :", status)

        with open(Reportfile,'wb') as export: ## Open temporary file as bytes
          export.write(response.content)
          print("\r", "This content will be ready shortly.")
      else:  
        print("\r","File",Reportfile,"not created.","Status:", status)
        Stat_data.append([w,status,format(time.time() - start_time, ".0f")])
      time.sleep(2)
     
    i +=1

# Create Status Sheet
E_status = pd.DataFrame(Stat_data, columns = ["Process Name","Status", "Execution Time"], index = None)
getdf= pd.DataFrame(E_status).to_excel('Execution Status.xlsx', index =False)



stat_wb = openpyxl.load_workbook('Execution Status.xlsx')
stat_ws = stat_wb['Sheet1']
stat_ws.column_dimensions['A'].width = 25
stat_ws.column_dimensions['B'].width = 20
stat_ws.column_dimensions['C'].width = 25
for c in stat_ws["1:1"]:
  c.alignment = Alignment(horizontal='center')

  c.font = Font(size=15, bold=True, color ='00FFFFFF' )
  c.fill = PatternFill("solid", fgColor="000066CC")
stat_wb.save('Execution Status.xlsx')
match_words.append('Execution Status.xlsx')



In [ ]:
#@title Clean report file for new PT


# delete old PT in Main Report
wb_del = openpyxl.load_workbook(new_template)


for sheetName in wb_del.sheetnames:
    if sheetName  in match_words :
        del wb_del[sheetName]
wb_del.save(new_template)



In [ ]:
#@title Change name of template with date ( refresh content or wait for the name to change before merging PTs)
master_file = new_template
temp_change = master_file.replace('.xlsx', '')
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M")
new_name = (temp_change +'_' + dt_string + '.xlsx')

file_out = os.rename(master_file,new_name)

In [ ]:

#@title Merging PTs to Report
path = os.getcwd()
files_dir = os.listdir(path)

files_PT = [f for f in files_dir if "KYR" in f]

files_PT.append('Execution Status.xlsx')

files_PT
p =0
while p < len(files_PT):
  l = files_PT[p]
  l_ws = [sheet.title for sheet in l]
  sheet_N = l.replace('.xlsx', '')

  wb = openpyxl.load_workbook(l)
  wb2 = openpyxl.load_workbook(new_name)

  sheetnames = wb.sheetnames
  for sheetname in sheetnames:
      print("Merging", sheet_N, "to Report")
      sheet = wb[sheetname]
      sheet2 = wb2.create_sheet(sheet_N)

# Color format
      sheet2.sheet_properties.tabColor = sheet.sheet_properties.tabColor

#Match cells between 2 sheets and merge.
      wm = list(sheet.merged_cells)
      if len(wm) > 0:
          for i in range(0, len(wm)):
              cell2 = str(wm[i]).replace('(<CellRange ', '').replace('>,)', '')
              sheet2.merge_cells(cell2)

      for i, row in enumerate(sheet.iter_rows()):
          sheet2.row_dimensions[i+1].height = sheet.row_dimensions[i+1].height
          for j, cell in enumerate(row):
              sheet2.column_dimensions[get_column_letter(j+1)].width = sheet.column_dimensions[get_column_letter(j+1)].width
              sheet2.cell(row=i + 1, column=j + 1, value=cell.value)

# Match the cell style
              source_cell = sheet.cell(i+1, j+1)
              target_cell = sheet2.cell(i+1, j+1)
              target_cell.fill = copy.copy(source_cell.fill)
              if source_cell.has_style:
                  target_cell._style = copy.copy(source_cell._style)
                  target_cell.font = copy.copy(source_cell.font)
                  target_cell.border = copy.copy(source_cell.border)
                  target_cell.fill = copy.copy(source_cell.fill)
                  target_cell.number_format = copy.copy(source_cell.number_format)
                  target_cell.protection = copy.copy(source_cell.protection)
                  target_cell.alignment = copy.copy(source_cell.alignment)

  
  wb2.save(new_name)
  wb.close()
  wb2.close()

  p +=1

print('Merging process completed. Final report is ready and will be downloaded automatically.')
print ( "The process costed : ",format(time.time() - start_time, ".0f")," seconds" )
files.download (new_name)